In [ ]:

import pandas as pd
from matplotlib import pyplot as plt
from pandas.plotting import autocorrelation_plot

from data_import import *


In [ ]:


def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')

series = pd.read_csv('shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
autocorrelation_plot(series)
plt.show()